# [Uninformed Search Algorithms](00_main_report.ipynb)

The informed search algorithms are a class of algorithms that use problem-specific knowledge to guide the search. 

$$ f(n) = g(n) + h(n) $$

where $f(n)$ is the total cost of the node, $g(n)$ is the cost to reach the node, and $h(n)$ is the heuristic cost of the node.

```mermaid
graph LR
A((Start)) --> B((Goal))
A --> D((D))
A --> E((E))
D --> F((F))
E --> F
F --> B
```

In this notebook, we will implement the following informed search algorithms:

- Depth First Search (DFS)
    - Depth Limited Search (DLS)
    - Iterative Deepening Search (IDS)
    - Recursive Depth Limited Search (RDLS)
- Breadth First Search (BFS)
- Uniform Cost Search  (UCS)

In [1]:
#  imports

### [Depth First Search (DFS)](https://en.wikipedia.org/wiki/Depth-first_search)

Is an algorithm for **traversing or searching tree or graph data structures**. The algorithm starts at the root node (selecting some arbitrary node as the root node in the case of a graph) and explores as far as possible along each branch before backtracking.

- $ Complexity: O(b^d) $
- $ Space: O(bd) $

where $b$ is the branching factor and $d$ is the depth of the goal node.

![dfs_01](./../resources/img/dfs_vs_bfs.gif)

In [7]:
def breadth_first_search(self, initial_state) -> None:
    """
    Breadth First Search algorithm
    """
    pass

#### Depth Limited Search (DLS)

Is a variant of depth-first search that limits the depth of the search tree. It is a recursive algorithm that uses a depth limit $l$ as a stopping condition. If the depth of the node exceeds the limit, the search is terminated.

##### Iterative (IDS or IDDFS)

Is a state space/graph search strategy in which a depth-limited version of depth-first search is run repeatedly with increasing depth limits until the goal is found.

IDDFS is optimal like breadth-first search, but **uses much less memory; at each iteration**, it visits the nodes in the search tree in the same order as depth-first search, but the cumulative order in which nodes are first visited is effectively breadth-firs

##### Recursive DLS


### [Breadth First Search (BFS)](https://en.wikipedia.org/wiki/Breadth-first_search)

Is an algorithm for **traversing or searching tree or graph data structures**.

It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key'[1]), and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

- Complexity: $ O(V+E) $
- Space: $ O(V) $

where $V$ is the number of vertices and $E$ is the number of edges.

![bfs_01](./../resources/img/bfs.gif)

In [8]:
def breadth_first_search(self, initial_state) -> None:
    """
    Breadth First Search algorithm
    """
    pass

### [Uniform Cost Search (UCS)](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)

This algorithm is very similar to Dijkstra's algorithm for finding the shortest path between nodes in a graph, but with a different **evaluation function** ("Dijsktra for large graphs...").

The UCS algorithm is a **greedy algorithm** that finds a **minimum spanning tree** for a weighted undirected graph.

Always evaluates the node with the **lowest path cost**.

- Complexity: $ O(b^{1+\frac{C*}{\epsilon}}) $
- Space: $ O(b^{1+\frac{C*}{\epsilon}}) $

where $b$ is the branching factor, $C*$ is the cost of the optimal solution, and $\epsilon$ is the accuracy of the heuristic.

![ucs_01](./../resources/img/ucs.gif)

In [9]:
def uniform_cost_search(self, initial_state) -> None:
    """
    Uniform Cost Search algorithm

    This algorithm is a special case of the A* algorithm
    
    """
    pass

#### Dijkstra's Algorithm

Is an algorithm for finding the shortest paths between nodes in a graph, which may represent, for example, road networks. It was conceived by computer scientist Edsger W. Dijkstra in 1956 and published three years later.

- Complexity: $ O(V^2) $
- Space: $ O(V) $

where $V$ is the number of vertices.

![dijkstra](./../resources/img/dijkstra.gif)

----
# Test It


- [some animation comparason](https://www3.cs.stonybrook.edu/~skiena/combinatorica/animations/search.html)